In [ ]:
!pip install pytube

     |████████████████████████████████| 56 kB 2.5 MB/s 


# Section 1. SETTING UP PARAMETERS

In [ ]:
SEARCH_LIST_1 = ['mặc gì', 'quần áo', 'xu hướng thời trang', 'phối đồ', 'tuần lễ thời trang', 'thiết kế thời trang']
SEARCH_LIST_2 = ['du học', 'dạy học', 'ôn thi', 'thi trung học phổ thông', 'giải đề thi','học trực tuyến']
SEARCH_LIST_3 = ['trò chơi điện tử','#gamethủ', '#thểthaođiệntử', 'trò chơi pc', 'trò chơi điện thoại', 'trò chơi trực tuyến', 'streamer trò chơi']
MAX_RESULT = 300

# Section 2. CLASS DEFINITION
© Ngô Phước Thịnh - 19520981

YoutubeScraper là class được viết dựa trên thư viện [pytube](https://pytube.io/en/latest/).

YoutubeScraper(search_list, maxResult):
* Default parameters:
  >search_list: ***list, default None***
  list các keyword.

  >maxResult: ***int, default 50*** 
  số kết quả trả về - *Lưu ý, con số này chỉ mang tính tương đối, ước lượng*. 

 


* YoutubeScraper.get_id(): hàm này sẽ trả về list ID (YoutubeCraper._id_list) của các video được tìm ra từ list các keyword tìm kiếm (Youtubescraper.search_list), dựa trên module Search của pytube. 
* YoutubeScraper.get_info(): dựa vào list video ID được tìm ra, hàm sẽ trả về title, description và keyword của các video trên (YotubeScraper._title_list, YoutubeScraper._keywords_list, YoutubeScraper._description_list). 

Ví dụ:

>```
>> yt = YoutubeScraper(["music", "trending"], 50)
>> yt.get_id()
>> yt.list_id
    ...
>> yt.get_info()
>> yt.list_title
    ...
```


*Lưu ý: Khi thu thập dữ liệu sẽ gửi số lượng lớn requests đến pytube và cả server Youtube, để giảm thiểu lỗi/server block và/hoặc các vấn đề khác liên quan đến các phương thức phòng vệ của server, các phương thức thu thập dữ liệu của class YoutubeScraper trên phải có thời gian delay giữa các lần request dữ liệu. Với keyword_list dài và số lượng kết quả lớn có thể tốn 3 - 5 giờ để hoàn thành. Ngoài ra, khả năng của thư viện pytube có hạn, do đó không nên đặt giá trị maxResult quá cao (>350).*


In [ ]:
from pytube import Search
from pytube import YouTube as Youtube
import time
import random
import pandas as pd

In [ ]:
class YoutubeScraper:

  def __init__(self, search_list, maxResult=50):
    self._id_list = []
    self._returned_list = []
    self.tag_list = []
    self._title_list = []
    self._keywords_list = []
    self._description_list = []
    self.search_list = search_list
    self.maxResult = maxResult//17*17

  def _single_search(self, tag): 
    s = Search(tag)
    while len(s.results) < self.maxResult:
      time.sleep(random.randint(5, 15))
      s.get_next_results()
    self.tag_list.extend([tag]*len(s.results))
    return s.results

  def _search(self):
    self._returned_list = []
    for tag in self.search_list:
      self._returned_list.append(self._single_search(tag))
      time.sleep(random.randint(5, 15))


  def get_id(self):
    self._search()
    for tag in self._returned_list:
      for obj in tag:
        self._id_list.append(str(obj)[-12:-1])

  @property
  def list_id(self):
    return self._id_list
  
  def _get_object(self, id):
    try:
      obj = Youtube('http://youtube.com/watch?v=' + id)
      return obj 
    except: 
      return None
  
  def _get_object_info(self, id):
    try:
      obj = self._get_object(id)
      return obj.title, obj.description, obj.keywords
    except:
      return None

  def get_info(self):
    for id in self._id_list:
      obj = self._get_object_info(id)
      if obj:
        time.sleep(random.randint(0, 25)/10)
        self._title_list.append(obj[0])
        self._description_list.append(obj[1])
        self._keywords_list.append(obj[2])
      else:
        self._title_list.append("")
        self._description_list.append("")
        self._keywords_list.append("")

  @property
  def list_title(self):
    return self._title_list
    
  @property
  def list_description(self):
    return self._description_list
    
  @property
  def list_keywords(self):
    return self._keywords_list

# Section 3. IMPLEMENT

In [ ]:
import pandas as pd

## 3.1. Fashion

In [ ]:
ytb_instance_1 = YoutubeScraper(SEARCH_LIST_1, MAX_RESULT)

In [ ]:
ytb_instance_1.get_id()

In [ ]:
ytb_instance_1.get_info()

In [ ]:
d={'Video ID': ytb_instance_1.list_id, 'Title': ytb_instance_1.list_title, 'Description': ytb_instance_1.list_description, 'Searchterm': ytb_instance_1.tag_list, 'Keywords': ytb_instance_1.list_keywords}
df = pd.DataFrame(data=d)

In [ ]:
df.to_csv('/content/drive/MyDrive/dataset/Youtube/YoutubeDataFashion.csv', encoding='utf-8',index=False)

## 3.2. Edu

In [ ]:
ytb_instance_2 = YoutubeScraper(SEARCH_LIST_2, MAX_RESULT)

In [ ]:
ytb_instance_2.get_id()

In [ ]:
ytb_instance_2.get_info()

In [ ]:
d={'Video ID': ytb_instance_2.list_id, 'Title': ytb_instance_2.list_title, 'Description': ytb_instance_2.list_description, 'Searchterm': ytb_instance_2.tag_list, 'Keywords': ytb_instance_2.list_keywords}
df = pd.DataFrame(data=d)

In [ ]:
df.to_csv('/content/drive/MyDrive/dataset/Youtube/YoutubeDataEducation.csv', encoding='utf-8',index=False)

## 3.3. Gaming

In [ ]:
ytb_instance_3 = YoutubeScraper(SEARCH_LIST_3, 300)

In [ ]:
ytb_instance_3.get_id()

In [ ]:
ytb_instance_3.get_info()

In [ ]:
d={'Video ID': ytb_instance_3.list_id, 'Title': ytb_instance_3.list_title, 'Description': ytb_instance_3.list_description, 'Searchterm': ytb_instance_3.tag_list, 'Keywords': ytb_instance_3.list_keywords}
df = pd.DataFrame(data=d)

In [ ]:
df.to_csv('/content/drive/MyDrive/dataset/Youtube/YoutubeDataGaming.csv', encoding='utf-8',index=False)

# Dataset Overview

In [ ]:
import pandas as pd

In [ ]:
edu = pd.read_csv("/content/drive/MyDrive/dataset/Youtube/YoutubeDataEducation.csv")

In [ ]:
gaming = pd.read_csv("/content/drive/MyDrive/dataset/Youtube/YoutubeDataGaming.csv")

In [ ]:
fashion = pd.read_csv("/content/drive/MyDrive/dataset/Youtube/YoutubeDataFashion.csv")

In [ ]:
edu.shape

(2096, 5)

In [ ]:
print(f'Dữ liệu thô về giáo dục có {edu.shape[0]} dòng')
print(f'Dữ liệu thô về thời trang có {fashion.shape[0]} dòng')
print(f'Dữ liệu thô về gaming có {gaming.shape[0]} dòng')

Dữ liệu thô về giáo dục có 2096 dòng
Dữ liệu thô về thời trang có 2100 dòng
Dữ liệu thô về gaming có 2078 dòng


In [ ]:
edu.head()

,Video ID,Title,Description,Searchterm,Keywords
0,KqwNoxWU-CM,Giáo dục 24/7: Cập nhật lịch đi học trực tiếp ...,BÁO LAO ĐỘNG |\nGiáo dục 24/7: Phân tích rủi r...,giáo dục,"['báo lao động', 'bao lao dong', 'bld', 'lao d..."
1,UL7JFReouNQ,"3 trụ cột Giáo Dục Hỏng , Đẩy học sinh vào đườ...",NaN,giáo dục,[]
2,GSIyO3IsQjM,Chữa Đề Tham Khảo Kỳ Thi Tốt Nghiệp THPT 2022│...,Chữa đề tham khảo (minh họa) thi tốt nghiệp TH...,giáo dục,"['#ChuademinhhoacuaBo', 'Chữa đề minh họa của ..."
3,dpEFsPbUUnE,Tư tưởng của Hồ Chí Minh về giáo dục | GS. Hoà...,Những câu chuyện về những người anh hùng dân t...,giáo dục,"['Đoàn thanh niên', 'Giáo viên', 'Học sinh', '..."
4,64Grw66GdoM,Phật Giáo Hòa Hảo - Một Nền Giáo Dục Phổ Thế,Chúng ta nhìn lại những người sáng lập các tôn...,giáo dục,"['giáo dục phổ thế', 'phật giáo hòa hảo', 'giá..."


In [ ]:
gaming_final = pd.DataFrame()
gaming_final['Title'] = gaming['Title']
gaming_final.loc[:,'Label'] = 'trò chơi'

In [ ]:
gaming_final.head()

,Title,Label
0,TÂM ĐIỂM TƯỚNG: BRIGHT [HƯỚNG DẪN CHƠI BRIGHT],trò chơi
1,TOP 1 Eland'orr | Đầu Game Bị Bán Hành Liệu Có...,trò chơi
2,Câu chuyện của Om Nom - Nibble Nom 💯 Trò chơi...,trò chơi
3,Game xưa - Super Mario - Ăn nấm cứu công chúa,trò chơi
4,Cứ nghĩ game khó ai ngờ…? #4 Trở Thành Siêu Pr...,trò chơi


In [ ]:
gaming_final.to_excel("/content/drive/MyDrive/Do_An Machine Learning 2022/Data_Check_Nhan/11_Gaming.xlsx")